## Hands-on

### Sentiment Analysis Using Naïve Bayes

In [4]:
#importing necessary libraries
import sys
import os
import re
import csv
from operator import itemgetter
import unicodedata

#importing classes
from codes.proc import Proc as P

### Phase 1
***
- ##### Data colect

In [3]:
filename = 'ODemonioeaSrtaPrym.pdf'
p = P(filename)

p.convertPDF()

name1 = 'Prym'
name2 = 'estrangeiro'

#Variable 'relation' is used to build training dataset
#relation = 'afinidade'
txt = p.get_text()
p.fname(txt,name1,name2)

"filename = 'ODemonioeaSrtaPrym.pdf'\np = P(filename)\n\np.convertPDF()\n\nname1 = 'Prym'\nname2 = 'Berta'\n\n#Variable 'relation' is used to build training dataset\n#relation = 'afinidade'\ntxt = p.get_text()\np.fname(txt,name1,name2)\n"

***
- ##### Preprocess

In this process we check if the `dataset of training` already exists, if not gets the `training base` and applies preprocess steps and generating `preprocessed-tb`.

In [2]:
w = []
with open('stopwords.txt') as f:
    sw = f.read().splitlines()

In [11]:
#look for stopwords
def common_elements(l1,l2):
    l = []
    for x in l1:
        if x in l2:
            l.append(x)
    return l

In [1]:
def preprocess(text):
    "remove accents"
    data = text
    normal = unicodedata.normalize('NFKD', data).encode('ASCII', 'ignore')
    text = (str(normal).replace('b\'','').replace('\'',''))

    "remove punctuation"
    text = ''.join(c for c in text if c not in punctuation)
    #row[1] = row[1].replace('-','')

    "lower case"
    lrow = text.split()
    lrow = [x.lower() for x in lrow]
    #lrow.pop(0) # remove primeiro elemento (nome)
    #lrow.pop() # remove ultimo elemento (nome)
    common = common_elements(sw,lrow)
    if len(common) > 0:
        for x in common:
            if x in lrow:
                lrow.remove(x)

    return ' '.join(str(x) for x in lrow)

In [10]:
#Training Base
tb = 'tb1.csv'
# Baseline dataset
base = 'base11.csv'

if not os.path.exists('./preprocessed-tb.csv'):
    with open('preprocessed-tb.csv', 'w', newline='') as outputfile:
        with open(tb, newline='') as csvfile:
            reader = csv.reader(csvfile, delimiter=',', quotechar='"')
            writer = csv.writer(outputfile, delimiter=',', quotechar='"')

            for row in reader:
                row[1] = preprocess(row[1])
                writer.writerow(row)
else:
    tb = 'preprocessed-tb.csv'

***
#### Load data


In [4]:
c = [] # [(classe, xapareceu)]
classes = []
tab = []
prob = []
P1 = []
P2 = []
n = 0

In [14]:
with open(tb, newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='"')
    for row in spamreader:
        n = n+1
        tab.append(row)
        #print('tab: ',tab)
        if row[0] not in classes:
            classes.append(row[0])
            c.append((row[0],1))
        else:
            for (i, j) in enumerate(c):
                if(j[0] == row[0]):
                    c[i] = ((row[0],c[i][1] + 1))


***
***
### Phase 2: Method

- ##### Prior Probability

In [6]:
print('Prior Probability')
if '' in classes:
    classes.remove('')

for x in classes:
    x = re.sub("[^a-zA-Z|^\s|^\t|^\r]+", "", x)
    for i in c:
        if i[0] == x:
            pi = i[1]/n
            break
        else:
            pi = 0
    P1.append((x,pi))
    
print(P1)

Prior Probability
[]


- ##### Constructing the Unigram Language Model

- ##### Posterior Probability

In [ ]:
for i,j in zip(half_left_list,half_right_list):
    if i[1] > j[1]:
        count_x = count_x + 1
    elif i[1] < j[1]:
        count_y = count_y + 1
    else:
        neutral = neutral + 1


print('Classification:', nv_class)
print(x+':', count_x, y+':', count_y)
print(f,nf)
print('f->',((f*100)/(nf+f)), 'nf->',(nf*100)/(nf+f))
